In [40]:
from tensorflow.keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt

In [41]:
from tensorflow.keras.utils import to_categorical

In [98]:
import tensorflow as tf

In [100]:
np.random.seed(42) == tf.random.set_seed(42)

True

In [42]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [43]:
train_images = np.array(train_images/255.0, dtype=np.float32)
train_labels = np.array(train_labels, dtype=np.float32)
test_images = np.array(test_images/255.0, dtype=np.float32)
test_labels = np.array(test_labels, dtype=np.float32)

In [59]:
train_oh_labels = to_categorical(train_labels)
test_oh_labels = to_categorical(test_labels)

In [55]:
train_oh_la.shape

NameError: ignored

# 모델 생성

* 함수형 API(Functional API)

In [46]:
from tensorflow.keras.models import Model

In [47]:
from tensorflow.keras.layers import Input, Flatten, Dense

In [48]:
input_tensor = Input(shape=(28, 28))
x = Flatten()(input_tensor) # 이런식으로 함수를 어떤 함수에 적용할 지 사용가능하다.
x = Dense(50, activation='sigmoid')(x)
output = Dense(10, activation='softmax')(x)



model = Model(inputs=input_tensor, outputs=output)

In [72]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                39250     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
Total params: 39,760
Trainable params: 39,760
Non-trainable params: 0
_________________________________________________________________


from tensorflow.keras.models import Sequential
model = Sequential([
                    Flatten(input_shape=(28, 28)),
                    Dense(100, activation='sigmoid'),
                    Dense(10, activation='softmax')
])
model.summary|

* 모델 컴파일 (optimizer, loss 등 설정)

In [50]:
from tensorflow.keras.optimizers import SGD

In [84]:
model.compile(optimizer=SGD(0.1), loss='categorical_crossentropy', metrics=['accuracy'])

## 모델 학습

* callback - ModelCheckpoint

In [115]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model

In [121]:
checkpoint_cb = ModelCheckpoint("my_keras_model.h5", save_best_only=True, verbose=1) 

In [122]:
#checkpoint_cb = ModelCheckpoint("weights.{epoch:02d}-{val_loss:.2f}.hdf5", save_best_only=True, verbose=1)

In [108]:
history = model.fit(train_images, train_oh_labels, validation_split=0.2, batch_size=128, epochs=20, verbose=1, callbacks=[checkpoint_cb]) #가장 좋은 성능을 my....h5로 저장한다.
#monitor에 따라서 mode가 min or max로 바꿔줘야 한다. 디폴트가 auto라서 신경 안써도 되는데 내부는 그러함

#ModelCheckpoint에서의 verbose탓에 improve되었는지 즉, 개선 여부를 알려줌

Epoch 1/20
375/375 [==============================] - 1s 4ms/step - loss: 0.2957 - accuracy: 0.8956 - val_loss: 0.3455 - val_accuracy: 0.8766

Epoch 00001: val_loss improved from inf to 0.34546, saving model to weights.01-0.35.hdf5
Epoch 2/20
375/375 [==============================] - 1s 3ms/step - loss: 0.2946 - accuracy: 0.8970 - val_loss: 0.3412 - val_accuracy: 0.8776

Epoch 00002: val_loss improved from 0.34546 to 0.34122, saving model to weights.02-0.34.hdf5
Epoch 3/20
375/375 [==============================] - 1s 3ms/step - loss: 0.2931 - accuracy: 0.8970 - val_loss: 0.3412 - val_accuracy: 0.8772

Epoch 00003: val_loss improved from 0.34122 to 0.34121, saving model to weights.03-0.34.hdf5
Epoch 4/20
375/375 [==============================] - 1s 3ms/step - loss: 0.2926 - accuracy: 0.8971 - val_loss: 0.3388 - val_accuracy: 0.8796

Epoch 00004: val_loss improved from 0.34121 to 0.33877, saving model to weights.04-0.34.hdf5
Epoch 5/20
375/375 [==============================] - 1s 3ms

In [95]:
model.evaluate(test_images, test_oh_labels, batch_size=128)

79/79 [==============================] - 0s 3ms/step - loss: 0.3902 - accuracy: 0.8606


[0.3902492821216583, 0.8605999946594238]

In [96]:
model2 = load_model('my_keras_model.h5')

In [97]:
model2.evaluate(test_images, test_oh_labels, batch_size=128)

79/79 [==============================] - 0s 2ms/step - loss: 0.3875 - accuracy: 0.8627


[0.38750794529914856, 0.8626999855041504]

* callback - EarlyStopping

In [111]:
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True)#가장 좋았던 시점의 가중치로 복구 # 10번을 기다림 좋아지지않으면 그만둔다.

In [114]:
history = model.fit(train_images, train_oh_labels, validation_split=0.2, batch_size=128, epochs=50, verbose=1, callbacks=[early_stopping_cb]) 

Epoch 1/50
375/375 [==============================] - 1s 3ms/step - loss: 0.2026 - accuracy: 0.9311 - val_loss: 0.3309 - val_accuracy: 0.8828
Epoch 2/50
375/375 [==============================] - 1s 3ms/step - loss: 0.2026 - accuracy: 0.9310 - val_loss: 0.3217 - val_accuracy: 0.8848
Epoch 3/50
375/375 [==============================] - 1s 3ms/step - loss: 0.2018 - accuracy: 0.9308 - val_loss: 0.3227 - val_accuracy: 0.8860
Epoch 4/50
375/375 [==============================] - 1s 3ms/step - loss: 0.2013 - accuracy: 0.9319 - val_loss: 0.3235 - val_accuracy: 0.8863
Epoch 5/50
375/375 [==============================] - 1s 3ms/step - loss: 0.2008 - accuracy: 0.9321 - val_loss: 0.3265 - val_accuracy: 0.8860
Epoch 6/50
375/375 [==============================] - 1s 3ms/step - loss: 0.2011 - accuracy: 0.9313 - val_loss: 0.3228 - val_accuracy: 0.8850
Epoch 7/50
375/375 [==============================] - 1s 3ms/step - loss: 0.2006 - accuracy: 0.9315 - val_loss: 0.3241 - val_accuracy: 0.8853
Epoch 

* Callback - ReduceLROnPlateau

In [117]:
reducelr_cb = ReduceLROnPlateau(verbose=1) #기울기에 따른 running rate 조절

In [119]:
history = model.fit(train_images, train_oh_labels, validation_split=0.2, batch_size=128, epochs=20, verbose=1, callbacks=[reducelr_cb])#여러개를 콜백할 수도 있다. 

Epoch 1/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1879 - accuracy: 0.9383 - val_loss: 0.3219 - val_accuracy: 0.8864
Epoch 2/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1879 - accuracy: 0.9377 - val_loss: 0.3214 - val_accuracy: 0.8855
Epoch 3/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1878 - accuracy: 0.9374 - val_loss: 0.3217 - val_accuracy: 0.8868
Epoch 4/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1878 - accuracy: 0.9374 - val_loss: 0.3216 - val_accuracy: 0.8867
Epoch 5/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1878 - accuracy: 0.9378 - val_loss: 0.3216 - val_accuracy: 0.8866
Epoch 6/20
375/375 [==============================] - 1s 4ms/step - loss: 0.1878 - accuracy: 0.9377 - val_loss: 0.3213 - val_accuracy: 0.8864
Epoch 7/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1877 - accuracy: 0.9376 - val_loss: 0.3215 - val_accuracy: 0.8863
Epoch 

In [120]:
model.evaluate(test_images, test_oh_labels, batch_size=128)

79/79 [==============================] - 0s 2ms/step - loss: 0.3460 - accuracy: 0.8799


[0.3460386395454407, 0.8798999786376953]

* callback 조합 reducelr, early_stop, checkpoint

In [123]:
history = model.fit(train_images, train_oh_labels, validation_split=0.2, batch_size=128, epochs=20, verbose=1, callbacks=[reducelr_cb, early_stopping_cb, checkpoint_cb])

Epoch 1/20
375/375 [==============================] - 1s 4ms/step - loss: 0.1864 - accuracy: 0.9386 - val_loss: 0.3213 - val_accuracy: 0.8873

Epoch 00001: val_loss improved from inf to 0.32127, saving model to my_keras_model.h5
Epoch 2/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1864 - accuracy: 0.9384 - val_loss: 0.3213 - val_accuracy: 0.8870

Epoch 00002: val_loss improved from 0.32127 to 0.32125, saving model to my_keras_model.h5
Epoch 3/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1864 - accuracy: 0.9383 - val_loss: 0.3213 - val_accuracy: 0.8871

Epoch 00003: val_loss did not improve from 0.32125
Epoch 4/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1863 - accuracy: 0.9381 - val_loss: 0.3213 - val_accuracy: 0.8867

Epoch 00004: val_loss did not improve from 0.32125
Epoch 5/20
375/375 [==============================] - 1s 3ms/step - loss: 0.1864 - accuracy: 0.9385 - val_loss: 0.3213 - val_accuracy: 0.8873

Epoch 0

In [124]:
model.evaluate(test_images, test_oh_labels, batch_size=128)

79/79 [==============================] - 0s 2ms/step - loss: 0.3460 - accuracy: 0.8801


[0.3460492193698883, 0.8801000118255615]

# 하이퍼 파라미터 튜닝

In [136]:
def build_model(n_neurons=50):
  input_tensor = Input(shape=(28, 28))
  x = Flatten()(input_tensor) # 이런식으로 함수를 어떤 함수에 적용할 지 사용가능하다.
  x = Dense(n_neurons, activation='sigmoid')(x)
  output = Dense(10, activation='softmax')(x)



  model = Model(inputs=input_tensor, outputs=output)
  model.compile(optimizer=SGD(0.1), loss='categorical_crossentropy', metrics=['accuracy'])
  
  return model

In [137]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor #텐서 안에도 사이킷런이 있다.

In [138]:
keras_clf = KerasClassifier(build_model)

In [139]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [144]:
parm_grid = {
    'n_neurons' : [50, 100]
}
grid_search = GridSearchCV(keras_clf, parm_grid, cv=3)
grid_search.fit(train_images, train_oh_labels, validation_split=0.2, batch_size=128, epochs=5, verbose=1)

Epoch 1/5
250/250 [==============================] - 54s 4ms/step - loss: 1.2725 - accuracy: 0.6634 - val_loss: 0.8577 - val_accuracy: 0.7412
Epoch 2/5
250/250 [==============================] - 1s 3ms/step - loss: 0.7563 - accuracy: 0.7582 - val_loss: 0.6809 - val_accuracy: 0.7673
Epoch 3/5
250/250 [==============================] - 1s 4ms/step - loss: 0.6411 - accuracy: 0.7843 - val_loss: 0.6076 - val_accuracy: 0.7896
Epoch 4/5
250/250 [==============================] - 1s 3ms/step - loss: 0.5817 - accuracy: 0.8024 - val_loss: 0.5703 - val_accuracy: 0.8006
Epoch 5/5
625/625 [==============================] - 1s 2ms/step - loss: 0.5357 - accuracy: 0.8135
Epoch 1/5
250/250 [==============================] - 1s 4ms/step - loss: 1.2677 - accuracy: 0.6521 - val_loss: 0.8610 - val_accuracy: 0.7510
Epoch 2/5
250/250 [==============================] - 1s 3ms/step - loss: 0.7613 - accuracy: 0.7587 - val_loss: 0.6852 - val_accuracy: 0.7648
Epoch 3/5
250/250 [==============================] - 1

GridSearchCV(cv=3, error_score=nan,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f370449bc10>,
             iid='deprecated', n_jobs=None, param_grid={'n_neurons': [50, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [146]:
grid_search.best_params_

{'n_neurons': 100}